In [ ]:
import pymongo
import os
import dotenv
import json
from openai import OpenAI

dotenv.load_dotenv()
dotenv.load_dotenv("../gorilla_openfunctions/.env")
openai_client = OpenAI(api_key = os.getenv("OPENAI_KEY_EMBEDINGS"))

In [ ]:
os.getenv("OPENAI_KEY_EMBEDINGS")

In [ ]:
_URI = f"mongodb://{os.environ['MONGODB_USERNAME']}:{os.environ['MONGODB_PASSWORD']}@{os.environ['MONGODB_HOST']}:{os.environ['MONGODB_PORT']}/{os.environ['MONGODB_DATABASE']}"
mogodb_client = pymongo.MongoClient(_URI)
raw_functions = mogodb_client["function_calling"]["raw_functions"]

In [ ]:
for function_desc in raw_functions.find():
    if "openai_embedding" in function_desc:
        continue
    func = function_desc["function"]
    func = json.dumps(func).replace("\n", " ")
    embedding = openai_client.embeddings.create(input=func, model="text-embedding-ada-002").data[0].embedding
    raw_functions.update_one({"_id": function_desc["_id"]}, {"$set": {"openai_embedding": embedding}})